In [2]:
import pandas as pd

# --- Load and Clean Data ---
file_path = 'April19_20.csv'
df = pd.read_csv(file_path, skiprows=3,dtype={'Phone2': str})
df = df[['Bill Date', 'OUTLET NAME', 'Customer Name', 'Phone2']]
df = df.iloc[1:].copy()

# Clean phone numbers
df['Phone2'] = df['Phone2'].astype(str).str.replace(r'\D+', '', regex=True).str.strip()
df['Customer Name'] = df['Customer Name'].astype(str).str.strip().str.upper()



# --- Define Conditions ---
def apply_conditions(df):
    df['NOB'] = 1  # Every row is one bill
    df['Repeting_More_3'] = (df['Phone2'] != '9091600916') & (df.groupby(['Bill Date', 'Phone2'])['Phone2'].transform('count') > 3)
    df['F2D_Cust_num'] = (df['Customer Name'] == 'FRESH2DAY') & (df['Phone2'] != '9091600916')
    df['Fake number'] = (df['Phone2'].str.len() != 10) | (~df['Phone2'].str[0].isin(['6', '7', '8', '9']))
    df['CustNum_WOT_F2D_name'] = (df['Phone2'] == '9091600916') & (df['Customer Name'] != 'FRESH2DAY')
    df['Fake_CustNum'] = df['Phone2'].str.startswith('90916009') & (~df['Phone2'].str[8:10].eq('16'))
    return df

df = apply_conditions(df)

# --- Create Summary Table ---
summary = df.groupby(['Bill Date', 'OUTLET NAME']).agg(
    NOB=('NOB', 'sum'),
    Repeting_More_3 = ('Repeting_More_3', 'sum'),
    F2D_Cust_num=('F2D_Cust_num', 'sum'),
    Fake_number=('Fake number', 'sum'),
    CustNum_WOT_F2D_name=('CustNum_WOT_F2D_name', 'sum'),
    Fake_CustNum=('Fake_CustNum', 'sum'),
).reset_index()

# Calculate percentages
for col in ['Repeting_More_3', 'F2D_Cust_num', 'Fake_number', 'CustNum_WOT_F2D_name', 'Fake_CustNum']:
    summary[f'{col} %'] = (summary[col] / summary['NOB']).round(4)

# --- Add Grand Total Row ---
grand = {
    'Bill Date': '',
    'OUTLET NAME': 'Grand Total',
    'NOB': summary['NOB'].sum()
}
for col in ['Repeting_More_3', 'F2D_Cust_num', 'Fake_number', 'CustNum_WOT_F2D_name', 'Fake_CustNum']:
    grand[col] = summary[col].sum()
    grand[f'{col} %'] = round(grand[col] / grand['NOB'], 4)

summary = pd.concat([summary, pd.DataFrame([grand])], ignore_index=True)

# --- Add Remarks Column to Main Data ---
def remark_row(row):
    reasons = []
    if row['Repeting_More_3']:
        reasons.append('Repeting_More_3')
    if row['F2D_Cust_num']:
        reasons.append('F2D_Cust_num')
    if row['Fake number']:
        reasons.append('Fake number')
    if row['CustNum_WOT_F2D_name']:
        reasons.append('CustNum_WOT_F2D_name')
    if row['Fake_CustNum']:
        reasons.append('Fake_CustNum')
    if (row['Customer Name'].strip().upper() == 'FRESH2DAY') and (row['Phone2'] == '9091600916'):
        reasons.append('Customer_care_num')
    return ', '.join(reasons) if reasons else 'Actual Customer'

df['Remark'] = df.apply(remark_row, axis=1)

# --- Export to Excel ---
output_file = 'violation_phone_summary.xlsx'
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    summary.to_excel(writer, sheet_name='Summary Report', index=False)
    df.to_excel(writer, sheet_name='Remarks', index=False)

    # Format percentage columns
    workbook = writer.book
    summary_ws = writer.sheets['Summary Report']
    percent_fmt = workbook.add_format({'num_format': '0.00%', 'align': 'center'})

    for col_idx, col in enumerate(summary.columns):
        if '%' in col:
            summary_ws.set_column(col_idx, col_idx, 18, percent_fmt)
        else:
            summary_ws.set_column(col_idx, col_idx, 18)

print(f"✅ Report saved to '{output_file}' with Summary and Remarks sheets.")


/var/folders/c_/8r42hlvx2w36c1j39xf03hkw0000gp/T/ipykernel_97590/3056099681.py:5: DtypeWarning: Columns (11,14,15,18,25,29,30,50,51) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, skiprows=3,dtype={'Phone2': str})


✅ Report saved to 'violation_phone_summary.xlsx' with Summary and Remarks sheets.
